In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import pymc as pm
import arviz as az

In [ ]:
# set seabron style and matplotlib parameters for very nice plots
sns.set_style('whitegrid')
mpl.rcParams['xtick.labelsize'] = 14
mpl.rcParams['ytick.labelsize'] = 14
mpl.rcParams['legend.title_fontsize'] = 14
mpl.rcParams['legend.fontsize'] = 12
mpl.rcParams['axes.labelsize'] = 16
mpl.rcParams['figure.figsize'] = [10,6]

# Bayesian model for inferring the mean of a dataset
This notebook introduces Bayesian inference with PyMC by taking the example of inferring the mean of a self-generated data set.
## Generate some data
First, we need some data. We can simply sample from a Normal distribution, because most things in psychology follow a Normal distribution. From these samples we want to infer the mean and the standard deviation. As we generated the data, we know the 'true' values of $\mu$ and $\sigma$ and can compare it with our inferred results. 

In [ ]:
# initialize random number generator (rng) of NumPy
rng = np.random.default_rng()

# sample data from Normal distribution
n = 1000 # number of data points
muTrue = 0 # mean of the normal distribution
sdTrue = 1 # standard deviation of the normal distribtion
data = rng.normal(muTrue, sdTrue, n) # sample random data

We can plot our samples as histogram and add the empirical mean and standard deviation:

In [ ]:
# plot histogram with empirical mean and sd
plt.hist(data, bins=30, density=True)
plt.vlines(data.mean(), 0, 1, colors='#d28140', label=f'$M$ = {data.mean().round(2)}')
plt.vlines([data.mean()-data.std(), data.mean()+data.std()], 0, 1, colors='#e6cdae', 
            label=f'$SD$ = {data.std().round(2)}')

plt.legend()
plt.show()

## Bayesian Workflow
The process of Bayesian modeling can be divided into three steps, also sometimes called Bayesian workflow:
1. <b>Design the model</b> based on the assumptions and how your data could have been generated.
2. <b>Bayesian inference</b> using Bayes' theorem to get the posterior distribution.
3. <b>Model checks</b> according to different criteria (e.g. convergence checks, expertise).

The first part is the most important and hardest part. Because you have to determine the relations between your parameters and variables, select appropriate probability distributions, and set the priors. The second part is solved by our probabilistic programming language (i.e. PyMC or any other PPL). The last part is also important, because we only have an approximation of the real posterior, and the checks try to ensure that we can trust our results.


# Design the model
One possible approach is to start with the data (i.e. dependent variable) and select the probability distribution that describes the data. Normally each probability distribution is defined by parameters (e.g. Normal: $\mu$ and $\sigma$) and as next step we have to define how these parameters are modeled. One possibility is to infer these parameters and select a probability distribution for each parameter again. Again these distributions have parameters, but now we can set these parameters to specific values. This would define our <font color=#8f3985><b>prior</b></font> and Bayesian inference infers a posterior for these. The other possibility would be that the parameters are defined by other variables and parameters. Here we would have to add these to our model too.

### <font color=#D28140><u>Exercise:</u></font>
How could the model for inferring the mean and standard deviation look like?

### Implement Bayesian models with PyMC
Now we have to translate our designed model into a PyMC model.

For model definitions in PyMC we can use a so-called <b>context manager</b>:

<code><b>with</b> pm.Model() <b>as</b> modelName:
    <i>code</i></code>
    
to collect all information (i.e. prior and likelihood) about our model. (Otherwise we have to link each variable to the model.)

We can define a parameter and the corresponding distribution with:

<code>parameter = pm.Distribution('parameterName', parameters of the distribution)</code>, e.g.

<code>mu = pm.Normal('mu', mu=0, sigma=1)</code>

For the <font color=#8f3985><b>Likelihood</b></font> we have to tell PyMC, that we observed data for this particular distribution, so we have to add <code>observed=data</code> after the parameter definitions, e.g. <code>pm.Normal(..., <b>observed=data</b>)</code>

### <font color=#D28140><u>Exercise:</u></font>
Implement the model for inferring the mean and standard deviation in PyMC. You can check yor model by just typing the name of the model in a code cell. (You can also print the model as graphical model with the graphviz package and <code>pm.model_graph.model_to_graphviz(model=modelName)</code>. You can install graphviz with <code>conda install -c conda-forge python-graphviz</code>.) 

## Prior Predictive Checks
As last check before we start our sampling or inference we can look at our <font color=#8f3985><b>prior predictive distribution</b></font>. This is the distribution of the data we expect according to our defined model and priors, before actually seeing any observed data. The process of sampling from the prior predictive distribution is called <font color=#8f3985><b>prior predictive checks</b></font>.

We can sample from the prior predictive distribution with <code>pm.sample_prior_predictive(model=modelName)</code> and then plot a histogram (or go for a KDE). For more complex cognitive models this step becomes really important and helpful. You can also look at the samples from priors. This can be helpful for priors that are created from other priors.

In [ ]:
priorPredictive = pm.sample_prior_predictive(model=meanModel)

In [ ]:
priorPredictive

You can access samples with: <code>az.extract(inferenceData, group, var_names)</code>. Some extracted data has more than one dimension and you have to convert it into a vector to plot a histogram. You can do this by adding <code>to_numpy().flatten()</code>.

In [ ]:
plt.figure()
plt.hist(az.extract(priorPredictive, group='prior', var_names='mu'), bins=100, density=True)
plt.title(f'Prior Predictive Distribution of $\mu$')
plt.show()

plt.figure()
plt.hist(az.extract(priorPredictive, group='prior', var_names='sd'), bins=100, density=True)
plt.title(f'Prior Predictive Distribution of $\sigma$')
plt.show()

plt.figure()
plt.hist(az.extract(priorPredictive, group='prior_predictive', var_names='Y').to_numpy().flatten(), bins=100, density=True)
plt.title(f'Prior Predictive Distribution of $Y$')
plt.show()

### <font color=#D28140><u>Exercise:</u></font>
Modify the priors and have a look how it changes the prior predictive distributions. Guess how it will change before plotting.

## Running the inference (i.e. sample the posterior)
After successful prior predictive checks we can start to infer the posterior. Hence, we can finally start to tell PyMC to do <font color=#8f3985><b>MCMC</b></font>-sampling to approximate our <font color=#8f3985><b>posterior</b></font> distributions. This is just one line of code <code>pm.sample()</code>! (You can also add this line after model definitions at the context manager.)

In [ ]:
inferenceData = pm.sample(model=meanModel)

As you can see, PyMC automatically selects an appropriate sampling algorithm (here NUTS), the number of chains (based on your hardware), and the number of samples. So PyMC does all the annoying stuff for us! (You can always change this default settings if you want or need to.)

## Inspect Posterior
PyMC gives us an inference data (iData) object as result, which contains the posterior samples (which are most interesting), but also some statistics, and the observed data:

In [ ]:
inferenceData

We can use <b>ArviZ</b> for plotting and statistics. (You find an overview here: https://python.arviz.org/en/stable/api/index.html) For instance, you can plot a KDE of the posterior (inclusive HDI) with <code>az.plot_posterior(iData, hdi_prob=HDI)</code> and get some statistics with <code>az.summary(iData)</code>. 

In [ ]:
az.plot_posterior(inferenceData, hdi_prob=.95);

In [ ]:
az.summary(inferenceData, hdi_prob=.95)

## MCMC Sampling diagnostics
The posterior distributions are approximations based on sampling and not exact calculations. Hence, before we start to interpret our posterior we have to check our sampling process. Just if we cannot find any hints that something went wrong while sampling, we can interpret the posterior with good conscience. To be able to understand the different diagnostic measures, it is important to remember how our posterior is inferred. Here is a quick reminder: we sample a markov chain that approximates the posterior distribution (i.e. with MCMC), so values with a higher posterior probability are sampled more often. Based on all samples we can create a histogram (or KDE) that shows the approximated posterior distribution. All samples are called the <b>trace</b>. While one trace can be made up of different <b>chains</b>. It is always clever to use more than one chain, because it is possible to get stuck in a local minima. As all chains have different start values we increase the probability to detect this problem.

### <font color=#D28140><u>Exercise:</u></font>
Go to https://bayesiancomputationbook.com/markdown/chp_02.html and choose one of the following dignostics: ESS, $\hat R$, or MCSE. Explain the concept of the diagnostic, calculate and / or plot it, and interpret the result.

### <font color=#495F75>Effective sample size (ESS)</font>

### <font color=#495F75>Potential scale reduction factor</font> $\hat{R}$

### <font color=#495F75>Monte Carlo standard error (MCSE)</font>

### <font color=#495F75>Trace plots</font>
<font color=#8f3985><b>Trace plots</b></font> show the sampled values and histogarms or KDEs for all chains. So we can check if all chains converged to the same distribution and we can see the strength of autocorrelation. The function is <code>az.plot_trace(posterior)</code>:

In [ ]:
az.plot_trace(inferenceData);

Ideally the distributions on the left should overlap, so all chains converged to the same distribution. And on the right we should see no trend, i.e. low autocorrelation.

### <font color=#495F75>Autocorrelation plots</font>
This is just another qualitative check of the autocorrelation, after we have quantified the autocorrelation with ESS. We can plot the autocorrelation for each chain and parameter with the function <code>az.plot_autocorr(posterior)</code>.

In [ ]:
az.plot_autocorr(inferenceData);

### <font color=#495F75>Rank plots</font>
This is also just another qualitative tool to compare the sampling behavior of the different chains. Here all samples from the trace are ranked and then the ranks are plotted for each chain separately. So if we have no bias we should have a uniform distribution, i.e. the same amount of samples from each chain within all rank bins. Get rank plots with <code>az.plot_rank(posterior)</code>.

In [ ]:
az.plot_rank(inferenceData);

### <font color=#495F75>Divergences</font>
<font color=#8f3985><b>Divergences</b></font> happen, when something went wrong during a sample, e.g. the sampler cannot sample a parameter value. You can get the number of divergent samples with: <code>iData.sample_stats.data_vars['diverging'].sum()</code>. If we have some divergences we can have a look at the pair plot with <code>az.plot_pair(posterior, divergences=True)</code> to (maybe) get a clue about the problem. If you have many divergences you probably should modify (i.e. <font color=#8f3985><b>reparameterize</b></font>) your model. (More about divergences: https://www.youtube.com/watch?v=6JDhM2MIYh0)

In [ ]:
inferenceData.sample_stats.data_vars['diverging'].sum()

In [ ]:
az.plot_pair(inferenceData, divergences=True);

## Posterior Predictive Checks (PPC)
As a final step we can use our posterior for sampling and can evaluate visually how close these samples from the inferred posterior match our actual data. <code>pm.sample_posterior_predictive(iData, model=modelName)</code> is the PyMC function for PPC:

In [ ]:
posteriorPredictive = pm.sample_posterior_predictive(inferenceData, model=meanModel)

In [ ]:
posteriorPredictive

In [ ]:
plt.figure()
plt.hist(az.extract(posteriorPredictive, group='posterior_predictive', var_names='Y').to_numpy().flatten(), label='PPC', density=True, bins=50)
plt.hist(data, label='Y', density=True, bins=50, alpha=.5) # alpha=.7 -> transparency
plt.legend()
plt.show()

###  <font color=#D28140><u>Exercises:</u></font>
- It's important to understand the influence of data, prior, and likelihood on posterior. Hence, check how the following modifications affect your posterior approximation. Always make a guess first, and make some notes (if you want to).
    - number of data points
    - parameters of prior
    - prior distribution
    - likelihood distribution
    - number of posterior samples
- Run convergence checks for some of your modified models.

<u>Hint:</u> You can create one cell with all the code you need or convert your notebook to a pure Python script, so you don't have to run all cells individually.

### <u>References:</u>
- Martin, O. A., Kumar, R., & Lao, J. (2022). <i>Bayesian Modeling and Computation in Python.</i> Chapman and Hall/CRC.

In [2]:
%load_ext watermark
%watermark -n -u -v -iv -w

Last updated: Mon Jun 17 2024

Python implementation: CPython
Python version       : 3.11.0
IPython version      : 8.20.0

seaborn   : 0.12.2
pymc      : 5.6.1
matplotlib: 3.8.4
arviz     : 0.17.1
numpy     : 1.25.2

Watermark: 2.3.1

